# Introduction to products and measurements
* **Compatability:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[`ga_ls5t_ard_3`](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3)

## Background
A "datacube" is a digital information architecture that specialises in hosting and cataloguing spatial information.
Digital Earth Australia (DEA) is based on the [Open Data Cube](https://www.opendatacube.org/) infrastructure, and specialises in storing remotely sensed data, particularly from Earth Observation satellites such as [Landsat](https://landsat.gsfc.nasa.gov/) and [Sentinel-2](https://www.copernicus.eu/en/about-copernicus/infrastructure/discover-our-satellites).

The DEA datacube contains both raw satellite data and derivative data "products".
These data products are often composed of a range of "measurements" such as the suite of remote sensing band values or statistical product summaries.

Therefore, before running a query to load data from the datacube, it is useful to know what it contains.
This notebook describes several straightforward ways to inspect the product and measurement contents of a datacube.

## Prerequisites
Users of this notebook should have a basic understanding of:
 - [How to run a Jupyter Notebook](Intro_to_Jupyter.ipynb)
 - [Satellite band naming and data processing naming conventions](Intro_to_DEA.ipynb)

## Description
This notebook demonstrates how to connect to a datacube and interrogate the available products and measurements stored within.

Topics covered include:
* How to connect to a datacube
* How to list all the products
* How to list all the product measurements
* How to interactively visualise data in the datacube 

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
The `datacube` package is required to access and work with available data.
The `pandas` package is required to format tables.
The `DcViewer` utility will allow us to interactively explore the products available in the datacube.

In [1]:
import datacube
import pandas as pd
from odc.ui import DcViewer

# Set some configurations for displaying tables nicely
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

### Connect to the datacube

After importing the `datacube` package, users need to specify a name for their session, known as the app name.

This name is generated by the user and is used to track down issues with database queries.
Use a short name that is consistent with the purpose of your notebook such as the way "Intro_to_products_and_measurements" has been used as the app name in this notebook.

The resulting `dc` object is what we use to access all the data contained within the Digital Earth Australia datacube.

In [2]:
# Temporary solution to account for Collection 3 data being in a different
# database on the NCI
try:
    dc = datacube.Datacube(app="Intro_to_products_and_measurements", env="c3-samples")
except:
    dc = datacube.Datacube(app="Intro_to_products_and_measurements")

## List products

Once a datacube instance has been created, users can explore the products and measurements stored within.

The following cell lists all products that are currently available in the DEA datacube by using the `dc.list_products()` function. 

Products listed under **name** in the following table represent the product options available when querying the datacube. 
The table below provides some useful information about each product, including a brief product **description**, the **instrument** and **platform** the data originated from (e.g. Landsat 8 OLI), and the product's default **crs** (coordinate reference system) and **resolution** if applicable.

For a comprehensive product description and access to complete product metadata, users are directed to the Geoscience Australia [Content Management Interface](https://cmi.ga.gov.au).



In [3]:
products = dc.list_products()
display_columns = [
    "name",
    "description",
    "platform",
    "instrument",
    "crs",
    "resolution",
]
products[display_columns].sort_index()

,name,description,platform,instrument,crs,resolution
id,,,,,,
1,ls8_nbart_geomedian_annual,"Surface Reflectance Geometric Median 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)",LANDSAT_8,OLI,EPSG:3577,"(-25, 25)"
2,ls7_nbart_geomedian_annual,"Surface Reflectance Geometric Median 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)",LANDSAT_7,ETM,EPSG:3577,"(-25, 25)"
3,ls5_nbart_geomedian_annual,"Surface Reflectance Geometric Median 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)",LANDSAT_5,TM,EPSG:3577,"(-25, 25)"
4,ls5_fc_albers,"Landsat 5 Fractional Cover 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)",LANDSAT_5,TM,EPSG:3577,"(-25, 25)"
5,ls8_fc_albers,"Landsat 8 Fractional Cover 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)",LANDSAT_8,OLI_TIRS,EPSG:3577,"(-25, 25)"
6,high_tide_comp_20p,High tide 20 percentage composites 25m v. 2.0.0,None,None,EPSG:3577,"(-25, 25)"
7,low_tide_comp_20p,Low tide 20 percentage composites 25m v. 2.0.0,None,None,EPSG:3577,"(-25, 25)"
8,ls8_barest_earth_albers,"Landsat-8 Barest Earth pixel composite albers 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)",LANDSAT_8,OLI,EPSG:3577,"(-25, 25)"
9,landsat_barest_earth,"Landsat-5/Landsat-7/Landsat-8 combined Barest Earth pixel composite albers 25 metre, 100km tile, Australian Albers Equal Area projection (EPSG:3577)","LANDSAT_5,LANDSAT_7,LANDSAT_8","TM,ETM+,OLI",EPSG:3577,"(-25, 25)"


## List measurements


Most products are associated with a range of available measurements.
These can be individual satellite bands (e.g. Landsat's near-infrared band) or statistical product summaries.

Using the **name** column of products listed above, let's interrogate the measurements associated with the `ga_ls5t_ard_3` product using the `dc.list_measurements()` function.
This product name refers to the Geoscience Australia Landsat 5 Analysis-ready data Collection 3 product.

The table below includes a range of technical information about each band in the dataset, including any **aliases** which can be used to load the data, the data type or **dtype**, any **flags_definition** that are associated with the measurement (this information is used for tasks like cloud masking), and the measurement's **nodata** value.

Change the `product` name below and re-run the following cell to explore available measurements associated with other products.

In [9]:
product = "s2a_ard_granule"

measurements = dc.list_measurements()
measurements.loc[product]

,aliases,dtype,flags_definition,name,nodata,spectral_definition,units
measurement,,,,,,,
azimuthal_exiting,[azimuthal_exiting],float32,NaN,azimuthal_exiting,-999,NaN,1
azimuthal_incident,[azimuthal_incident],float32,NaN,azimuthal_incident,-999,NaN,1
exiting,[exiting],float32,NaN,exiting,-999,NaN,1
incident,[incident],float32,NaN,incident,-999,NaN,1
relative_azimuth,[relative_azimuth],float32,NaN,relative_azimuth,-999,NaN,1
relative_slope,[relative_slope],float32,NaN,relative_slope,-999,NaN,1
satellite_azimuth,[satellite_azimuth],float32,NaN,satellite_azimuth,-999,NaN,1
satellite_view,[satellite_view],float32,NaN,satellite_view,-999,NaN,1
solar_azimuth,[solar_azimuth],float32,NaN,solar_azimuth,-999,NaN,1


## Visualising available data
For a more visual way of exploring the data that is available within the Digital Earth Australia datacube, we can use the interactive `DcViewer` utility. 
Select a product from the drop-down menu on the top-right of the map to show the areas data is available for in blue.
You can also use the back and forward buttons above the map to toggle through time.

The utility is only able to visualise a limited number of datasets at one time.
If the available data footprints do not appear, either press the "show" button on the top right, or zoom in on the map.

In [5]:
DcViewer(dc=dc, 
         time='2015', 
         width='800px',
         center=(-27.48, 153.10),
         zoom=7)

### DEA Explorer sites
Another way to view the available data within a datacube is to visit the DEA Datacube Explorer sites.
These webpages visualise the data that is available for every product in DEA.

There is a different Datacube Explorer page for both the NCI and DEA Sandbox environments to account for the different datasets that are available in each of these datacubes:

1. [NCI DEA Datacube Explorer](http://explorer.dea.ga.gov.au)
2. [DEA Sandbox Datacube Explorer](http://explorer.sandbox.dea.ga.gov.au)

![DEA Explorer](../Supplementary_data/Introduction_to_products_and_measurements/DEAExplorer.JPG)

## Recommended next steps

To continue working through the notebooks in this beginners guide, the following notebooks are designed to be worked through in the following order:
1. [Introduction to Jupyter](Intro_to_products_and_measurements.ipynb)
2. [Introduction to Digital Earth Australia](Intro_to_DEA.ipynb)
3. Introduction to Products and Measurements (this notebook)
4. [Introduction to Querying](Intro_to_querying.ipynb)
5. [Introduction to Plotting](Intro_to_plotting.ipynb)
6. Run a basic analysis (in progress)
7. Other training materials (in progress)


Once you have worked through the beginners guide, you can join advanced users by exploring:
- [DEA datasets](../DEA_datasets)
- [Frequently used code](https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Frequently_used_code)
- [Real world examples](https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Real_world_examples)

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** October 2019

**Compatible `datacube` version:** 

In [6]:
print(datacube.__version__)

1.7+128.gebdc898a.dirty


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)